In [ ]:
# From Coursera Julia course by university of Cape Town
using Pkg;
# ENV["PYTHON"]="/home/maruthinh/softwares/julia-1.10.0-alpha1/bin/julia"
# Pkg.build("PyCall")
Pkg.add("PyPlot")
Pkg.add("Distributions") #create random variables
Pkg.add("StatsBase") #Basic statistical support
Pkg.add("CSV") #Reading and writing CSV files
Pkg.add("DataFrames") #create a data structure 
Pkg.add("HypothesisTests") #create a statistical tests
Pkg.add("StatsPlots") #plot statistical data
Pkg.add("GLM") #General linear models

In [ ]:
using CSV
using DataFrames
using Distributions
using GLM
using HypothesisTests
using PyPlot
using StatsBase
using StatsPlots
# pyplot()

#### Creating random variables

In [ ]:
age = rand(18:80, 100); #uniform distribution
wcc = round.(rand(Distributions.Normal(12,2), 100), digits=1); #white cell count. Normal distribution & round to one decimal place 
crp = round.(Int, rand(Distributions.Chisq(4), 100)) .* 10; #Chi-squared distribution with broadcasting & alternative round() : C-reactive protein
treatment = rand(["A", "B"], 100); #uniformly weighted
result = rand(["Improved" "Static" "Worse"], 100); #uniformly weighted

#### Desriptive statistics

In [ ]:
#mean of age 
mean(age)

In [ ]:
#median of age 
median(age)

In [ ]:
#standard deviation
std(age)

In [ ]:
#variance of age 
var(age)

In [ ]:
#Descriptive statistics of the age variable 
StatsBase.describe(age)

In [ ]:
#Summary stats of the wcc variable (similar to describe)
StatsBase.summarystats(wcc)

#### Creating a dataframe
create a dataframe data for easier manipulation

In [ ]:
data = DataFrame(Age=age, WCC=wcc, CRP=crp, Treatment=treatment, Result=result)

In [ ]:
#number of rows and cols 
size(data)

In [ ]:
#first row 
first(data)

In [ ]:
#extract data frame based on condition
dataA = data[data.Treatment .== "A", :]
dataB = data[data.Treatment .== "B", :]

##### Descriptive stat using the dataframe 

In [ ]:
describe(data)

In [ ]:
# groupby() function in the DataFrames package is used to group rows in a DataFrame based on the unique values in one or more columns
# Now, grouped_df is a GroupedDataFrame object, which contains groups of rows grouped by unique values in the "ID" column.
grouped_df = groupby(data, :Age)
# You can iterate over the groups 
for group in grouped_df
    println(group)
end

In [ ]:
# You can perform various operations on groups, such as calculating summary statistics (e.g., mean, sum) or applying custom functions using combine() or map() functions.
mean_scores = combine(grouped_df, :WCC => mean => :WCC)

### Visualizing the data
Plots package with DataFrame macro 

In [ ]:
@df data density(:Age, group=:Treatment, title="Distribution of ages by treatment group", xlab="Age", ylab="Distribution", legent=:topright)

In [ ]:
#we can do the same for thre resulting groups 
@df data density(:Age, group = :Result, title="Distribution of ages by result group", xlab="Age", ylab="Distribution", legent=:topright)

In [ ]:
#discriminate between all groups 
@df data density(:Age, group=(:Treatment, :Result), title="Distribution of ages by treatment and result group", xlab="Age", ylab="Distribution", legent=:topright)


In [ ]:
#create a box-and-whisker plot of the white cell per treatment group and then per result group
@df data StatsPlots.boxplot(:Treatment, :WCC, lab="WCC", title="white cell count by treatment group", xlab="Groups", ylab="WCC")

In [ ]:
@df data StatsPlots.boxplot(:Result, :WCC, lab="Result", title="white cell count by result group", xlab="Groups", ylab="WCC")

In [ ]:
#correlation between numerical variables using a correlation plot and corner plot. 
@df data corrplot([:Age :WCC :CRP], grid=false) #no comma between arguments 

In [ ]:
@df data cornerplot([:Age :WCC :CRP], grid=false, compact=true) #no comma between arguments 

### Inferential statistics

In [ ]:
# We will begin by Student's t test 
#difference in age between patients in groups A and B
HypothesisTests.EqualVarianceTTest(dataA.Age, dataB.Age)

In [ ]:
#p value for the difference in white cell count between patients in groups A and B 
pvalue(EqualVarianceTTest(dataA.WCC,dataA.WCC))

In [ ]:
#difference in c-reactive protein level between patients in groups A and B for unequal variances 
UnequalVarianceTTest(dataA.CRP, dataB.CRP)

### Creating a liner models using GLM 

In [ ]:
#Simple model to predict CRP 
GLM.fit(LinearModel, @formula(CRP~1), data)

In [ ]:
#Adding age as a predictor variable 
fit(LinearModel, @formula(CRP~Age), data)

In [ ]:
#Adding age and WCC as a predictor variable 
fit(LinearModel, @formula(CRP~Age+WCC), data)

We can conduct $\chi^2$ test for independence using the HypothesisTests.ChisqTest() function. First we need to look at the counts. Below we calculate the number of unique values for the result variable sample spaces for patients in groups A and B

In [ ]:
groupby(dataA, :Result)
groupby(dataB, :Result)

observed=reshape([22,17,18,18,11,14],(2,3))
ChisqTest(observed)

### Export to CSV

In [ ]:
CSV.write("data.csv", data)